In [3]:
import math

Write wind speed to energy (efficiency calculation) function (Laura)
- input: average windspeed for a station (mph).
- mph to kmh conversion
- kmh  to kWh conversion (probably)
- multiply by efficiency factor.

#### Things to consider:
- nameplate capacity of newly installed wind turbines in the US in 2018 was 2.43 MW (is this commercial)
- average rotor diameter is 115.6 meters
- does power coefficient change with wind speed?

https://www.energy.gov/sites/prod/files/2019/08/f65/2018%20Wind%20Technologies%20Market%20Report%20FINAL.pdf
**Average turbine capacity, rotor diameter, and hub height increased in 2018, continuing the longterm trend.**
- To optimize wind project cost and performance, turbines continue to grow in size. The average rated (nameplate) capacity of newly installed wind turbines in the United States in 2018 was 2.43 MW, up 5% from the previous year and 239% since 1998−1999. The average rotor diameter in 2018 was 115.6 meters, a 2% increase over 2017 and 141% over 1998−1999, while the average hub height in 2018 was 88.1 meters, up 2% over the previous year and 57% since 1998−1999.

The amount of energy generated by a wind turbine depends on:
- wind speed (main factor)
- the area swept by the blades
- air density

Wind turbines require:
- a minimum wind speed (generally 12-14 km/h) to begin turning and generate electricity
- strong winds (50-60 km/h) to generate at full capacity
- winds of less than 90 km/h; beyond that speed, the turbines must be stopped to avoid damage

$P_{avail} = 0.5 \rho A v^{3} C_{p}$

Units: 
- rho (kg/m^3): air density
- A(m^2): area of the wind turbine blade
- v (m/s): wind speed
- Cp(unitless): capacity factor (efficiency)

In [12]:
# I think the actual argument for this function would need to be the wind df
# rho is air density, which has minimal change at the height of a wind turbine (~90 m). It changes from
# 1.225 kg/m^3 to 1.215


def wind_power(wind_speed):
    """
    This function calculates the available power from a commercial wind turbine based on 
    wind speed.
    
    Argument: wind speed (in mph).
    
    Returns: If power avaiable is greater tahn the max conversion for the wind turbine, the
    amount returned is the 
    """
    mph = wind_speed
    kph = wind_speed * 1.61
    v = kph/(3.6)
    rho = 1.225
    area = math.pi*(115.6/2)**2
    Cp = 0.419
    
    np_capacity = 2.43 * 1e3
    
    if kph < 12.0 or kph > 90.0:
        print('The location you have inputted cannot support wind energy')
    else:
        pass
   
    
    p_avail = (0.5 * rho * area * (v**3) * Cp)/1e3
    
    if p_avail > np_capacity:
        return round(np_capacity, 2)
    else:
        return round(p_avail, 2)

In [13]:
wind_power(30.0)

2430.0

In [14]:
def wind_kwh(wind_speed):
    power = wind_power(wind_speed)
    
    kwh = (power * 365 * 24)
    
    return kwh

In [15]:
wind_kwh(55.0)

21286800.0